Theorems (or conjectures) for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum.QPE</a>
========

In [ ]:
import proveit
# 72 cols ==============================================================
# Prepare this notebook for defining the theorems of a theory:
%theorems_notebook # Keep this at the top following 'import proveit'.
from proveit import a, b, e, k, l, m, n, r, t, u, U, eps
from proveit import IndexedVar, ExprRange,  Lambda
from proveit.linear_algebra import MatrixMult, ScalarMult, SU, TensorProd, VecAdd, VecSum
from proveit.logic import Implies, And, Equals, Forall, InSet, NotEquals, NotInSet, SubsetEq, Difference
from proveit.numbers import Abs, Add, Neg, subtract, Exp, exp, frac, Mult, Div, sqrt, sqrd, Sum
from proveit.numbers import greater, greater_eq, Less, LessEq, Mod
from proveit.statistics import Prob
from proveit.numbers import zero, one, two, three, four
from proveit.numbers import Interval, IntervalCC, IntervalCO, IntervalOO, IntervalOC
from proveit.numbers import Integer, IntegerNonZero, IntegerNeg, Natural, NaturalPos, Complex, Real, RealNonNeg
from proveit.numbers import i, pi
from proveit.physics.quantum import Bra, Ket, Meas, NumKet, Qmult
from proveit.physics.quantum import ket0, ket1, inv_root2
from proveit.physics.quantum.QPE import (
    _u, u_tilde, _n, _t, phase, _phase, _phase_est, phase_est, _m,
    _b, _delta, two_pow_t, _two_pow_t, _two_pow_t_minus_one, _alpha_l, _alpha_l_sqrd,
    _diff_l_scaled_delta, _full_domain, _full_domain_sans_zero, _neg_domain,
    _pos_domain, _e_domain, _U_pow_two_pow_k,
    _Psi, _psi, _psi_t, _psi__t, m_ket_domain, two_pow_m,
    phase_est_circuit, success_prob_guarantee, t_req)
from proveit.physics.quantum.QPE.phase_est_ops import Psuccess, Pfail, ModAdd
from proveit.physics.quantum.QPE.phase_est_ops import SubIndexed
from proveit.physics.quantum.QFT import InverseFourierTransform
from proveit.trigonometry import Sin
from IPython.display import display

In [ ]:
%begin theorems

#### Some convenience methods for building expressions:

In [ ]:
def exp2pi_i(*exp_factors):
    return exp(Mult(*((two, pi, i) + exp_factors)))

def exp2pi_i_on_two_pow_t(*exp_factors):
    return exp(frac(Mult(*((two, pi, i) + exp_factors)), _two_pow_t))

def exp_neg_2pi_i_on_two_pow_t(*exp_factors):
    return exp(Neg(frac(Mult(*((two, pi, i) + exp_factors)), _two_pow_t)))

display(exp2pi_i(a, b))
display(exp2pi_i_on_two_pow_t(a, b))
display(exp_neg_2pi_i_on_two_pow_t(a, b))

## Local theorems (for convenience - used only internally)

#### Take care of number domain issues:

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
# Thus, 2^t is a positive natural number
_two_pow_t_is_nat_pos = InSet(_two_pow_t, NaturalPos)

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
_two_pow_t_minus_one_is_nat_pos = InSet(Exp(two, subtract(_t, one)), NaturalPos)

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
_two_pow_t_less_one_is_nat_pos = InSet(subtract(_two_pow_t, one), NaturalPos)

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
_two_pow_t_not_zero = NotEquals(_two_pow_t, zero)

In [ ]:
# The o-plus addition denotes addition modulo 2^t, resulting in an integer
_mod_add_closure = Forall((a, b), InSet(ModAdd(a, b), 
                                        Interval(zero, subtract(Exp(two, _t), one))), 
                         domain=Integer)

In [ ]:
# The phase phi is in the real interval [0, 1)
_phase_is_real = InSet(_phase, Real)

In [ ]:
# the subscripts here should be inside the Kets!
_psi_t_expansion = Forall(
    t,
    Equals(NumKet(_psi, Add(t, one)),
           TensorProd(Add(Ket(zero), ScalarMult(Exp(e, Mult(two, pi, _phase, i, Exp(two, t) )) ,Ket(one) )), NumKet(_psi, t))),
    domain=NaturalPos)

In [ ]:
psi_prime_expansion = Forall(
    t,
    Equals(NumKet(_psi, Add(t, one)),
           TensorProd(Ket(p_prime_t), NumKet(psi_prime, t))),
    domain=NaturalPos)

In [ ]:
# Need variable t instead of literal t
# also need linear algebra terms ScalarMult and VecAdd in several places
_psi_t_as_tensor_prod = Forall(
    t, Equals(
        Ket(_psi_t),
        Mult(
            frac(one, Exp(two, frac(_t, two))),
            TensorProd(ExprRange(r, Add(ket0, Mult(exp(Mult(two, pi, i, _phase, Exp(two, Neg(r)))), ket1)), 
                                 Neg(subtract(_t, one)), zero)))),
    domain=NaturalPos)

In [ ]:
# Notice we're using a ScalarMults instead of QMults on the rhs
# at least until QMults are automatically simplified to do that instead
# -------------------------------------------------------------------------- #
# This theorem is meant to capture Nielsen & Chuang's formula 5.20 (pg 222). #
# -------------------------------------------------------------------------- #
_psi_t_var_formula = Forall(
    t, Equals(Ket(_psi_t),
              ScalarMult(frac(one, Exp(two, frac(t, two))),
                    VecSum(k, ScalarMult(exp(Mult(two, pi, i, _phase, k)), NumKet(k, t)),
                           domain=Interval(zero, subtract(Exp(two, t), one))))),
    domain=NaturalPos)

In [ ]:
# psi_t_lit_formula with _t (literal _t) instead of variable t
# _psi_t_formula = Forall(
#     t, Equals(Ket(_psi_t),
#               Qmult(frac(one, Exp(two, frac(_t, two))),
#                     VecSum(k, Qmult(exp(Mult(two, pi, i, _phase, k)), NumKet(k, _t)),
#                            domain=Interval(zero, subtract(Exp(two, _t), one))))),
#     domain=NaturalPos)

In [ ]:
# apply an axiomatic definition here (see QPE axioms)
_Psi_via_psi = Equals(Ket(_Psi), Qmult(InverseFourierTransform(_t),
                                       Ket(_psi__t)))

In [ ]:
_best_is_int = InSet(_b, Integer)

In [ ]:
_pos_domain_in_full_domain = Forall(
        e, Forall(l, InSet(l, _full_domain), domain=_pos_domain),
        domain=NaturalPos)

In [ ]:
_pos_domain_in_full_domain_sans_zero = Forall(
        e, SubsetEq(_pos_domain, _full_domain_sans_zero),
        domain=_e_domain)

In [ ]:
_neg_domain_in_full_domain = Forall(
        e, Forall(l, InSet(l, _full_domain), domain=_neg_domain),
        domain=NaturalPos)

In [ ]:
_neg_domain_in_full_domain_alt = Forall(
        e, SubsetEq(_neg_domain, _full_domain),
        domain=NaturalPos)

In [ ]:
_neg_domain_in_full_domain_sans_zero = Forall(
        e, SubsetEq(_neg_domain, _full_domain_sans_zero),
        domain=_e_domain)

In [ ]:
_pos_domain_within_integer = Forall(
        e, SubsetEq(_pos_domain, Integer),
        domain=_e_domain)

In [ ]:
_neg_domain_within_integer = Forall(
        e, SubsetEq(_neg_domain, Integer),
        domain=_e_domain)

In [ ]:
_delta_is_real = InSet(_delta, Real)

#### This derives from $\delta$ being the difference between $\delta$ and its best $t$-bit estimate (without going over):

In [ ]:
_scaled_delta_in_interval = InSet(Mult(_two_pow_t, _delta), IntervalCO(zero, one))

In [ ]:
_success_prob_is_real = Forall(e, InSet(Psuccess(e), Real), domain=NaturalPos)

In [ ]:
_all_alpha_l_is_complex = Forall(l, InSet(_alpha_l, Complex), domain=Integer)

In [ ]:
_all_abs_alpha_l_are_nonneg = Forall(
    l, InSet(Abs(_alpha_l), RealNonNeg),
    domain=Integer)

In [ ]:
_all_abs_alpha_l_sqrd_are_real = Forall(
    l, InSet(Exp(Abs(_alpha_l), two), Real),
    domain=Integer)

#### Follows from scaled_delta_in_interval:

In [ ]:
_scaled_delta_not_eq_nonzeroInt = Forall(
        l, NotEquals(Mult(_two_pow_t, _delta), l),
        domain=Integer, conditions = [NotEquals(l, zero)])

In [ ]:
_delta_not_eq_scaledNonzeroInt = Forall(
        l, NotEquals(_delta, frac(l, _two_pow_t)),
        domain=Integer, conditions = [NotEquals(l, zero)])

In [ ]:
_delta_diff_in_interval = Forall(
        l,
        InSet(subtract(_delta, frac(l, _two_pow_t)),
              IntervalCO(Neg(frac(one, two)), frac(one, two))),
        domain=_full_domain)

In [ ]:
_scaled_delta_diff_in_interval = Forall(
        l,
        InSet(Mult(two, pi, subtract(_delta, frac(l, _two_pow_t))),
              IntervalCC(Neg(pi), pi)),
        domain=_full_domain)

In [ ]:
_non_int_delta_diff = Forall(
        l,
        NotInSet(subtract(_delta, frac(l, _two_pow_t)),
                Integer), 
        domain=_full_domain,
        conditions = [NotEquals(l, zero)])

In [ ]:
# use alternative below instead of this one?
# Notice the condition in the alternate that l ≠ 0
_scaled_abs_delta_diff_interval = Forall(
        l,
        InSet(Mult(pi, Abs(subtract(_delta, frac(l, _two_pow_t)))),
              IntervalOC(zero, Div(pi, two))),
        domain=_full_domain)

In [ ]:
_scaled_abs_delta_diff_interval_alt = Forall(
        l,
        InSet(Mult(pi, Abs(subtract(_delta, frac(l, _two_pow_t)))),
              IntervalOC(zero, Div(pi, two))),
        domain=_full_domain,
        conditions = [NotEquals(l, zero)])

#### *Success probability as sum of individual success event probabilities:*

In [ ]:
_m_from_meas = Equals(_m, Meas(Ket(_Psi)))

In [ ]:
_phase_est_from_meas = Equals(Mult(_two_pow_t, _phase_est), 
                              Meas(Ket(_Psi)))

In [ ]:
# don't yet know what's happening here — steps are all working but something still missing
# What is now missing? Appears to be an issue with m vs. 2^t phi — HC
# _success_sum = Forall(
#         e,
#         greater_eq(Psuccess(e),
#                   Sum(l, Prob(Equals(Mult(_two_pow_t, _phase_est), ModAdd(_b, l)), 
#                               _phase_est), 
#                       domain=Interval(Neg(e), e))),
#         domain=NaturalPos)

In [ ]:
# REWRITING: Now modifying to put back the m instead of 2^t phi
_success_sum = Forall(
        e,
        greater_eq(Psuccess(e),
                  Sum(l, Prob(Equals(_m, ModAdd(_b, l)), 
                              _m), 
                      domain=Interval(Neg(e), e))),
        domain=NaturalPos)

#### *Failure probability as sum of individual failure event probabilities in terms of $\alpha_l$, amplitude of $\lvert \Psi \rangle$ for a state specified relative to $b$ (the best outcome state):*

In [ ]:
_fail_sum = Forall(
        e,
        LessEq(Pfail(e),
               Add(Sum(l, _alpha_l_sqrd, domain=_neg_domain),
                   Sum(l, _alpha_l_sqrd, domain=_pos_domain))),
        domain=_e_domain)

#### *Modulo addition may be converted to regular addition within $2 \pi i$ exponentiation:*

In [ ]:
_exp2pi_i_modadd = Forall(
        (a, b),
        Equals(exp2pi_i_on_two_pow_t(ModAdd(a, b)), 
               exp2pi_i_on_two_pow_t(Add(a, b))),
        domain=Integer)

#### *Direct evaluation of $\alpha_l$ (via an intermediate step first):*

In [ ]:
_alpha_l_eval = Forall(
        l,
        Equals(_alpha_l,
               Mult(frac(one, _two_pow_t),
                   Sum(k, Mult(exp_neg_2pi_i_on_two_pow_t(k, ModAdd(_b, l)),
                               exp2pi_i(_phase, k)),
                       domain=Interval(zero, subtract(_two_pow_t, one))))),
        domain=Integer)

#### *Evaluation of $\alpha_l$ after performing the geometric series summation in terms of $\delta$:*

In [ ]:
_phase_from_best = Equals(_phase, Add(frac(_b, _two_pow_t), _delta))

In [ ]:
_scaled_delta_minus_l__in__real = Forall(l, InSet(subtract(Mult(_two_pow_t, _delta), l), Real),
                                         domain = Integer)

In [ ]:
_delta_diff_exp_not_one = Forall(
    l,
    NotEquals(exp2pi_i(subtract(_delta, frac(l, _two_pow_t))),
          one),
    domain=_full_domain,
    conditions = [NotEquals(l, zero)])

In [ ]:
_alpha_l_summed = Forall(
    l,
    Equals(_alpha_l,
           Mult(frac(one, _two_pow_t),
                    frac(subtract(one, exp2pi_i(subtract(Mult(_two_pow_t, _delta), l))),
                         subtract(one, exp2pi_i(subtract(_delta, frac(l, _two_pow_t))))))),
    domain=_full_domain,
    conditions = [NotEquals(l, zero)])

In [ ]:
_alpha_l_summed_abs = Forall(
    l,
    Equals(Abs(_alpha_l),
           frac(Abs(subtract(one,
                             exp(Mult(two,pi,i,
                                      subtract(Mult(_two_pow_t,_delta),l))))),
                Mult(_two_pow_t,
                     two,
                     Sin(Mult(pi, Abs(subtract(_delta, frac(l, _two_pow_t)))))))),
    domain=_full_domain,
    conditions = [NotEquals(l, zero)])

#### *$| \alpha_l |^2$ inequality to bound the failure probability:*

In [ ]:
_l_non_zero = Forall(l, InSet(l, IntegerNonZero), domain=_full_domain_sans_zero)

In [ ]:
_alpha_l_sqrd_ineq = Forall(
    l,
    LessEq(_alpha_l_sqrd,
           frac(one,
                Mult(four, Exp(_diff_l_scaled_delta, two)))),
    domain=_full_domain_sans_zero)

#### *A bound on the failure probability:*

In [ ]:
_pos_domain_within_natpos = Forall(e, SubsetEq(_pos_domain, NaturalPos), domain=_e_domain)

In [ ]:
_neg_domain_within_negint = Forall(e, SubsetEq(_neg_domain, IntegerNeg), domain=_e_domain)

In [ ]:
_fail_ineq_lemma = Forall(
    e,
    LessEq(Pfail(e), 
           Mult(frac(one, four), 
                Add(Sum(l, frac(one, sqrd(_diff_l_scaled_delta)), domain=_neg_domain),
                    Sum(l, frac(one, sqrd(_diff_l_scaled_delta)), domain=_pos_domain)))), 
    domain=_e_domain)

In [ ]:
_fail_ineq = Forall(
    e,
    LessEq(Pfail(e), Mult(frac(one,two), Add(frac(one,e),
                                             frac(one, Exp(e, two))))), 
    domain=_e_domain)

### The externally usable theorems (as opposed to local theorems)

In [ ]:
qpe_quarantee = Forall(
    eps, Forall(
        (m, n, t), Forall(
            U, Forall(
                (u, u_tilde), Forall(
                    (phase, phase_est),
                    Implies(phase_est_circuit,
                            success_prob_guarantee)
                    .with_wrap_before_operator(),
                    domain=Real,
                    condition=Equals(Qmult(U, Ket(u)),
                                     exp2pi_i(phase))),
                domain=m_ket_domain),
            domain=SU(two_pow_m)),
        domain=NaturalPos,
        condition=Equals(t, t_req)),
    domain=IntervalOO(zero, one))

In [ ]:
qpe_eigenvec = Forall(
    [m, t], Forall(
        U, Forall(
            [u, u_tilde], Forall(
                (phase, phase_est),
                Implies(phase_est_circuit,
                        Equals(Ket(u), Ket(u_tilde)))
                .with_wrap_before_operator(),
                domain=Real,
                condition=Equals(Qmult(U, Ket(u)),
                                 exp2pi_i(phase))),
            domain=m_ket_domain),
        domain=SU(two_pow_m)),
    domain=NaturalPos)

In [ ]:
%end theorems